In [95]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [96]:
file_path = tf.keras.utils.get_file("shakespeare.txt", "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")
text = open(file_path, 'rb').read().decode(encoding='utf-8')

In [97]:
len(text)

1115394

In [98]:
characters = sorted(set(text))

In [99]:
len(characters)

65

In [100]:
char_to_index = dict((c, i) for i, c in enumerate(characters))

In [101]:
char_to_index

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'K': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'W': 35,
 'X': 36,
 'Y': 37,
 'Z': 38,
 'a': 39,
 'b': 40,
 'c': 41,
 'd': 42,
 'e': 43,
 'f': 44,
 'g': 45,
 'h': 46,
 'i': 47,
 'j': 48,
 'k': 49,
 'l': 50,
 'm': 51,
 'n': 52,
 'o': 53,
 'p': 54,
 'q': 55,
 'r': 56,
 's': 57,
 't': 58,
 'u': 59,
 'v': 60,
 'w': 61,
 'x': 62,
 'y': 63,
 'z': 64}

In [102]:
index_to_char = dict((i, c) for i, c in enumerate(characters))

In [103]:
index_to_char

{0: '\n',
 1: ' ',
 2: '!',
 3: '$',
 4: '&',
 5: "'",
 6: ',',
 7: '-',
 8: '.',
 9: '3',
 10: ':',
 11: ';',
 12: '?',
 13: 'A',
 14: 'B',
 15: 'C',
 16: 'D',
 17: 'E',
 18: 'F',
 19: 'G',
 20: 'H',
 21: 'I',
 22: 'J',
 23: 'K',
 24: 'L',
 25: 'M',
 26: 'N',
 27: 'O',
 28: 'P',
 29: 'Q',
 30: 'R',
 31: 'S',
 32: 'T',
 33: 'U',
 34: 'V',
 35: 'W',
 36: 'X',
 37: 'Y',
 38: 'Z',
 39: 'a',
 40: 'b',
 41: 'c',
 42: 'd',
 43: 'e',
 44: 'f',
 45: 'g',
 46: 'h',
 47: 'i',
 48: 'j',
 49: 'k',
 50: 'l',
 51: 'm',
 52: 'n',
 53: 'o',
 54: 'p',
 55: 'q',
 56: 'r',
 57: 's',
 58: 't',
 59: 'u',
 60: 'v',
 61: 'w',
 62: 'x',
 63: 'y',
 64: 'z'}

In [117]:
seq_length = 40
step_size = 2

sentences = []
next_characters = []

for i in range(0, len(text) - seq_length, step_size):
    
    sentences.append(text[i: i+seq_length])
    next_characters.append(text[i+seq_length])

In [118]:
x = np.zeros((len(sentences), seq_length, len(characters)), dtype=np.bool8)
y = np.zeros((len(sentences), len(characters)), dtype=np.bool8)

In [119]:
for i, sentence in enumerate(sentences):
    
    for t, character in enumerate(sentence):
        x[i, t, char_to_index[character]] = 1
    
    y[i, char_to_index[next_characters[i]]] = 1

In [120]:
model = Sequential()

model.add(LSTM(64, input_shape= (seq_length, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation("softmax"))

model.compile(loss = "categorical_crossentropy",
              optimizer = RMSprop(learning_rate= 0.01))

model.fit(x, y, batch_size=128, epochs=5)

Epoch 1/5
4357/4357 [==============================] - 109s 25ms/step - loss: 1.8835
Epoch 2/5
4357/4357 [==============================] - 112s 26ms/step - loss: 1.6674
Epoch 3/5
4357/4357 [==============================] - 114s 26ms/step - loss: 1.6238
Epoch 4/5
4357/4357 [==============================] - 114s 26ms/step - loss: 1.6022
Epoch 5/5
4357/4357 [==============================] - 114s 26ms/step - loss: 1.5900


In [121]:
#model.save("Text_Generator.model")

In [122]:
#model = tf.keras.models.load_model("Text_Generator.model")

In [123]:
def sample(preds, temperature=1.0):
    
    preds= np.asarray(preds).astype("float64")
    preds= np.log(preds) / temperature
    exp_preds= np.exp(preds)
    preds= exp_preds / np.sum(exp_preds)
    probas= np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [124]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - seq_length - 1)
    generated = ''
    sentence = text[start_index: start_index + seq_length]
    generated += sentence
    
    for i in range(length):
        x_predictions = np.zeros((1, seq_length, len(characters)))
        
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions,
                            temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
        
    return generated

In [125]:
print("----------0.2----------")
print(generate_text(300, 0.2))

----------0.2----------
 sepulchre!'
With this, we charged against the command thee the counter thee the man thee the pray
the country's hand the senter the commander
That the country the brother have the country's the shame.

CORIOLANUS:
What the man that the man that the man thee the country's that the man,
And the country the with the cause the man thee and s


In [126]:
print("----------0.4----------")
print(generate_text(300, 0.4))

----------0.4----------
INCENTIO:

LUCIO:
'Faith, my lord. I spout thee men a commise
And state his that his be an the father,
Of the man to the strengther worth and the country,
That dishon so losg be strange, and that have some than the man and the will thee than other.

BUCKINGHAM:
And that that I that many the son I will be the
receive the wife you that what


In [127]:
print("----------0.6----------")
print(generate_text(300, 0.8))

----------0.6----------
 soon
hot, my very lips might freeze to the om it
Whose of I brother not; for fellowing brother
thour lait; and 'low thee hope me was my gurse
hardet so comes mare now many sweeth.
But not this repore tell you Romeo, you live that
Or king that shall be on, what is hence.

GREMIONE:
Go therefore is them then a
fine, the purpost the holy on


In [128]:
print("----------0.8----------")
print(generate_text(300, 0.8))

----------0.8----------
fty in our commonwealth:
All must be ever have so pare here time;
And the father.

Servose,--true?'

OLrigE:
Them than you never incomets, I will ours in Marenat
In heal my wrathers of indeal of heaven
our handed viot-breat thee at marross,
Rou brother to sake as I came to him thee indeat
Thee seeminny where be only our from now.

LUCENTI


In [129]:
print("----------1----------")
print(generate_text(300, 1))

----------1----------
blessed, blessed night! I am afeard.
Beinger. and accemphed: you tell the dost;
And is wor have way set notering himself.
For he as yhowreties he have cannot this man,
And follivenes indeed wot you. 
LUCINIUS:
thou Clapeip nog more clain!

ARIAR:
Glander.

DUKE VINCENTIO:
Thou art alongen untovertal, you loves that-
Whom me weepes in ang 
